# --- Day 6: Chronal Coordinates ---
The device on your wrist beeps several times, and once again you feel like you're falling.

"Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."

The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.

If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the largest distance from the other points.

Using only the Manhattan distance, determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate (and aren't tied in distance to any other coordinate).

Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:

```
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
```
If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:
```
..........
.A........
..........
........C.
...D......
.....E....
.B........
..........
..........
........F.
```
This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:
```
aaaaa.cccc
aAaaa.cccc
aaaddecccc
aadddeccCc
..dDdeeccc
bb.deEeecc
bBb.eeee..
bbb.eeefff
bbb.eeffff
bbb.ffffFf
```
Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.

In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

What is the size of the largest area that isn't infinite?

In [136]:
test1 = [(1, 1),
         (1, 6),
         (8, 3),
         (3, 4),
         (5, 5),
         (8, 9)]
filein = 'input6.txt'
with open(filein) as f:
    data = list(map(lambda x: tuple(int(i) for i in x.split(',')), f.read().splitlines()))
f.close()


In [155]:
test = False
if test:
    coords = test1
else:
    coords = data

In [156]:
#helper
def mdist(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return abs(x2-x1)+abs(y2-y1)

In [157]:
from collections import defaultdict

#find bounding box
xsortCoords = sorted(coords,key=lambda c: c[0])
ysortCoords = sorted(coords,key=lambda c: c[1])
xmin,xmax = xsortCoords[0][0],xsortCoords[-1][0]
ymin,ymax = ysortCoords[0][1],ysortCoords[-1][1]

grid = [[dict() for i in range(ymin,ymax+1)] for j in range(xmin,xmax+1)]
gridsimple = [[None for i in range(ymin,ymax+1)] for j in range(xmin,xmax+1)]
offset = (xmin,ymin)
coord_index = list(map(lambda p: (p[0]-offset[0],p[1]-offset[1]),coords))

# populate matrix with distances
i = 0
maxd = 0
for c in coord_index:
    x,y = c
    for j in range(len(grid)):
        for k in range(len(grid[0])):
            d = mdist(c,(j,k))
            grid[j][k][i] = d
            if d> maxd:
                maxd = d
    i+=1

    # find the closest areas
idCounts = defaultdict(int)
for j in range(len(grid)):
    for k in range(len(grid[0])):
        lowVal = maxd+1
        for key,value in grid[j][k].items():
            if value < lowVal:
                lowVal = value
                id = key
            elif value == lowVal:
                id = '.'
        gridsimple[j][k] = id
        idCounts[id]+=1

# Find largest area not touching the edge (ie: not infinite)
maxarea = 0
for key,value in idCounts.items():
    if key == '.':
        continue
        
    c = coord_index[key]
    if (c[0] == xmin) or (c[0] == xmax) or (c[1] == ymin) or (c[1] == ymax):
        continue
    
    if value > maxarea:
        maxarea = value


In [158]:
gridsimple
maxarea

4887

# --- Part Two ---
On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region near as many coordinates as possible.

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:
```
..........
.A........
..........
...###..C.
..#D###...
..###E#...
.B.###....
..........
..........
........F.
```
In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:
```
Distance to coordinate A: abs(4-1) + abs(3-1) =  5
Distance to coordinate B: abs(4-1) + abs(3-6) =  6
Distance to coordinate C: abs(4-8) + abs(3-3) =  4
Distance to coordinate D: abs(4-3) + abs(3-4) =  2
Distance to coordinate E: abs(4-5) + abs(3-5) =  3
Distance to coordinate F: abs(4-8) + abs(3-9) = 10
Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30
```
Because the total distance to all coordinates (30) is less than 32, the location is within the region.

This region, which also includes coordinates D and E, has a total size of 16.

Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.

What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [162]:
gridsum = [[0 for i in range(ymin,ymax+1)] for j in range(xmin,xmax+1)]
if test:
    dlimit = 32
else:
    dlimit = 10000

# find the summed distances
safearea = 0
for j in range(len(grid)):
    for k in range(len(grid[0])):
        for key,value in grid[j][k].items():
            gridsum[j][k] += value
        
        if gridsum[j][k] < dlimit:
            safearea += 1
        else:
            gridsum[j][k] = '.'
    

In [163]:
gridsum
safearea

34096